<a href="https://colab.research.google.com/github/s2t2/tweet-analysis-2020/blob/news/app/news/User_Details_20210806_Exporting_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:
#
# SETUP GOOGLE DRIVE
#

import os
from google.colab import drive

drive.mount('/content/drive')
print(os.getcwd(), os.listdir(os.getcwd())) #> 'content', ['.config', 'drive', 'sample_data']

DIRPATH = '/content/drive/My Drive/NeuralNetworkOpinions/Impeachment'
print(DIRPATH)
os.path.isdir(DIRPATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgQSexiG9NDznxgdGdQh5gWyYapVzjqZK_PH1W-QYESKiBBdXFwZaw
Mounted at /content/drive
/content ['.config', 'drive', 'sample_data']
/content/drive/My Drive/NeuralNetworkOpinions/Impeachment


True

In [4]:
#
# BIG QUERY CREDENTIALS
#

# google.cloud checks the file at path designated by the GOOGLE_APPLICATION_CREDENTIALS env var
# so we set it here using the shared credentials JSON file from our shared google drive
# and verify it for good measure

GOOGLE_CREDENTIALS_FILEPATH = os.path.join(DIRPATH, "credentials", "tweet-research-shared-268bbccc0aac.json") 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_CREDENTIALS_FILEPATH

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") # implicit check by google.cloud
print(GOOGLE_APPLICATION_CREDENTIALS) # verification for implicit check
print(os.path.isfile(GOOGLE_APPLICATION_CREDENTIALS)) # verification for implicit check

/content/drive/My Drive/NeuralNetworkOpinions/Impeachment/credentials/tweet-research-shared-268bbccc0aac.json
True


In [5]:
#
# BIG QUERY SERVICE
#

from google.cloud import bigquery
from pandas import DataFrame

class BigQueryService():
    def __init__(self):
        self.client = bigquery.Client()

    def execute_query(self, sql, verbose=True):
        if verbose == True: 
            print(sql)
        job = self.client.query(sql)
        return job.result()
    
    def query_to_df(self, sql, verbose=True):
        records = [dict(row) for row in list(self.execute_query(sql, verbose=verbose))]
        return DataFrame(records)



# Data Dictionary




column_name | datatype | description
--- | --- | ---
user_id	| INTEGER | unique identifier for each user in our "impeachment 2020" dataset
created_on	| DATE | date the user was created
screen_name_count	| INTEGER | number of screen names used 
screen_names	| STRING | all screen names used
is_bot	| BOOLEAN | whether or not we classified this user as a "bot" / automated account
bot_rt_network	| INTEGER | for bots, which retweet network (0:anti-trump, 1:pro-trump)
is_q	| BOOLEAN | whether or not this user tweeted Q-anon language / hashtags
q_status_count	| INTEGER | the number of tweets with Q-anon language / hashtags
status_count	| INTEGER | number of total tweets authoried by this user (in our "impeachment 2020" dataset only)
rt_count	| INTEGER | number of total retweets authoried by this user (in our "impeachment 2020" dataset only)
avg_score_lr	| FLOAT | avergage opinion score from our Logistic Regression model (0:anti-trump, 1:pro-trump)
avg_score_nb	| FLOAT | avergage opinion score from our Naive Bayes model (0:anti-trump, 1:pro-trump)
avg_score_bert	| FLOAT | avergage opinion score from our BERT Transformer model (0:anti-trump, 1:pro-trump) 
opinion_community	| INTEGER | binary classification of average opinion (0:anti-trump, 1:pro-trump)
follower_count	| INTEGER | number of followers (in our "impeachment 2020" dataset only)
follower_count_b	| INTEGER | ... who are bots
follower_count_h	| INTEGER | ... who are humans
friend_count	| INTEGER | number of friends (in our "impeachment 2020" dataset only)
friend_count_b	| INTEGER | ... who are bots
friend_count_h	| INTEGER | ... who are humans
avg_toxicity	| FLOAT | average "toxicity" score from the Detoxify model
avg_severe_toxicity	| FLOAT | average "sever toxicity" score from the Detoxify model
avg_insult	| FLOAT | average "insult" score from the Detoxify model
avg_obscene	| FLOAT | average "obscene" score from the Detoxify model
avg_threat	| FLOAT | average "threat" score from the Detoxify model
avg_identity_hate	| FLOAT | average "identity hate" score from the Detoxify model
urls_shared_count (TODO)	| INTEGER | number of tweets with URLs in them (TODO)
fact_scored_count	| INTEGER | number of tweets with URL domains that we have rankings for
avg_fact_score	| FLOAT | average fact score of links shared (1: fake news, 5: mainstream media)

# CSV Pipeline 

In [10]:
from pandas import read_csv

DESTRUCTIVE = True

csv_filepath = os.path.join(DIRPATH, "user_details_20210806.csv")

if os.path.isfile(csv_filepath) and not DESTRUCTIVE:
    print("FOUND EXISTING CSV FILE!")
    df = read_csv(csv_filepath)
else:
    print("GENERATING NEW CSV FILE!")
    bq_service = BigQueryService()
    sql = f"""
        SELECT *
        FROM `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u
    """
    df = bq_service.query_to_df(sql)
    df.to_csv(csv_filepath, index=False)


GENERATING NEW CSV FILE!

        SELECT *
        FROM `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u
    


In [9]:
print(len(df))
df.head()

3600545


,Unnamed: 0,user_id,created_on,screen_name_count,screen_names,is_bot,bot_rt_network,is_q,q_status_count,status_count,rt_count,avg_score_lr,avg_score_nb,avg_score_bert,opinion_community,follower_count,follower_count_b,follower_count_h,friend_count,friend_count_b,friend_count_h,avg_toxicity,avg_severe_toxicity,avg_insult,avg_obscene,avg_threat,avg_identity_hate,fact_scored_count,avg_fact_score
0,0,1201029281482235904,2019-12-01,1,HARRIETSUSAN4,False,NaN,False,NaN,18,16,0.0556,0.1111,0.0046,0,66.0,2.0,64.0,343.0,6.0,337.0,0.050687,0.000232,0.002113,0.001177,0.000461,0.003759,0,NaN
1,1,337091850,2011-07-17,1,SUSANMMRAZ,False,NaN,False,NaN,105,80,0.0962,0.0667,0.0747,0,1072.0,49.0,1023.0,1742.0,120.0,1622.0,0.115957,0.001983,0.058983,0.049420,0.000667,0.004879,2,3.892857
2,2,2370334990,2014-02-28,1,EDDIETHUNDER44,False,NaN,False,NaN,35,15,0.0857,0.0286,0.0637,0,1199.0,40.0,1159.0,1761.0,95.0,1666.0,0.129561,0.008591,0.058599,0.039499,0.009718,0.004566,0,NaN
3,3,858913836,2012-10-03,1,RICHARDCRESSE,True,NaN,True,1.0,700,681,0.9186,0.9257,0.9826,1,202.0,18.0,184.0,503.0,78.0,425.0,0.063513,0.000393,0.015775,0.012956,0.000578,0.001460,8,1.625000
4,4,622750531,2012-06-30,1,YERBOUTISHEIK,False,NaN,False,NaN,184,154,0.7772,0.8315,0.8826,1,2423.0,75.0,2348.0,1613.0,93.0,1520.0,0.120060,0.005381,0.050481,0.064212,0.002673,0.003119,1,2.642857
